In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import class_weight

In [2]:
data_dir = "/content/drive/MyDrive/Dataset of Tuberculosis Chest X-rays Images"
IMAGE_SIZE = 224
BATCH_SIZE = 32

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 3018 files belonging to 2 classes.


In [4]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,validation_split=0.2)

train_data = train_datagen.flow_from_directory(data_dir, target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                               batch_size=BATCH_SIZE, class_mode='binary',
                                               subset='training',shuffle=True)
test_data = train_datagen.flow_from_directory(data_dir, target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                               batch_size=BATCH_SIZE, class_mode='binary',
                                               subset='validation',shuffle=False)

Found 2416 images belonging to 2 classes.
Found 602 images belonging to 2 classes.


In [5]:
lables = dataset.class_names
lables

['Normal', 'TB']

In [6]:
weights = class_weight.compute_class_weight(class_weight='balanced',
                                            classes=np.unique(lables),
                                            y=lables)

class_weights = dict(enumerate(weights))
print(class_weights)

{0: np.float64(1.0), 1: np.float64(1.0)}


In [7]:
base_model = tf.keras.applications.VGG16(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),
                                               include_top=False,
                                               weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False
x= tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)

prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

model.summary()


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,241,025 (58.14 MB)

 Trainable params: 526,337 (2.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [8]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
              tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [9]:
history = model.fit(train_data,validation_data=test_data,
          epochs=6,
          class_weight=class_weights)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/6
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.8287 - loss: 0.3575 - precision: 0.8599 - recall: 0.9463

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


76/76 ━━━━━━━━━━━━━━━━━━━━ 890s 12s/step - accuracy: 0.8294 - loss: 0.3562 - precision: 0.8604 - recall: 0.9466 - val_accuracy: 0.9684 - val_loss: 0.1244 - val_precision: 0.9859 - val_recall: 0.9760
Epoch 2/6
76/76 ━━━━━━━━━━━━━━━━━━━━ 16s 213ms/step - accuracy: 0.9539 - loss: 0.1301 - precision: 0.9654 - recall: 0.9800 - val_accuracy: 0.9801 - val_loss: 0.0744 - val_precision: 0.9900 - val_recall: 0.9860
Epoch 3/6
76/76 ━━━━━━━━━━━━━━━━━━━━ 16s 213ms/step - accuracy: 0.9781 - loss: 0.0704 - precision: 0.9846 - recall: 0.9892 - val_accuracy: 0.9850 - val_loss: 0.0547 - val_precision: 0.9940 - val_recall: 0.9880
Epoch 4/6
76/76 ━━━━━━━━━━━━━━━━━━━━ 16s 214ms/step - accuracy: 0.9806 - loss: 0.0599 - precision: 0.9883 - recall: 0.9878 - val_accuracy: 0.9917 - val_loss: 0.0425 - val_precision: 0.9960 - val_recall: 0.9940
Epoch 5/6
76/76 ━━━━━━━━━━━━━━━━━━━━ 20s 207ms/step - accuracy: 0.9878 - loss: 0.0414 - precision: 0.9917 - recall: 0.9935 - val_accuracy: 0.9850 - val_loss: 0.0425 - val_

In [10]:
model.save('final_model.h5')